In [1]:
import sys
import shutil
import nltk
from pyspark import SparkConf, SparkContext
from nltk.tokenize import word_tokenize
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1574368456847_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

invalid syntax (<stdin>, line 11)
  File "<stdin>", line 11
    from pyspark.mllib.tree import DecisionTree, DecisionTreeModel/
                                                                  ^
SyntaxError: invalid syntax



In [3]:
from pyspark import SparkConf, SparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import nltk

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
nltk.download('punkt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

I/O operation on closed file
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/nltk/downloader.py", line 835, in download
    % (msg.package.id, download_dir)
  File "/usr/local/lib/python3.6/site-packages/nltk/downloader.py", line 783, in show
    subsequent_indent=prefix + prefix2 + ' ' * 4,
  File "/tmp/8091970019160777614", line 534, in write
    super(UnicodeDecodingStringIO, self).write(s)
ValueError: I/O operation on closed file



In [20]:
from nltk.tokenize import word_tokenize

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.mllib.feature import HashingTF, IDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
pos = sc.textFile("s3://bucketzr14/pos.txt")

neg = sc.textFile("s3://bucketzr14/neg.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
pos_sp = pos.flatMap(lambda line: line.split("\n")).collect()

neg_sp = neg.flatMap(lambda line: line.split("\n")).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
all_words = []

documents = []

allowed = ["J", "R", "V", "N"]

for p in pos_sp:
    documents.append({"text": p , "label": 1})

for p in neg_sp:
    documents.append({"text": p , "label": 0})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
raw_data = sc.parallelize(documents)

raw_tokenized = raw_data.map(lambda dic : {"text": dic["text"] , "label" : dic["label"]})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
htf = HashingTF(50000)

raw_hashed = raw_tokenized.map(lambda dic : LabeledPoint(dic["label"], htf.transform(dic["text"])))

raw_hashed.persist()

trained_hashed, test_hashed = raw_hashed.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
NB_model = NaiveBayes.train(trained_hashed)

NB_prediction_and_labels = test_hashed.map(lambda point : (NB_model.predict(point.features), point.label))

NB_correct = NB_prediction_and_labels.filter(lambda predicted, actual: predicted == actual)

#NB_accuracy = NB_correct.count() / float(test_hashed.count())

#print ("NB training accuracy:" + str(NB_accuracy * 100) + " %")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
LR_model = LogisticRegressionWithLBFGS.train(trained_hashed)

LR_prediction_and_labels = test_hashed.map(lambda point : (LR_model.predict(point.features), point.label))

LR_correct = LR_prediction_and_labels.filter(lambda predicted, actual: predicted == actual)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
SVM_model = SVMWithSGD.train(trained_hashed, iterations = 10)

SVM_prediction_and_labels = test_hashed.map(lambda point : (SVM_model.predict(point.features), point.label))

SVM_model.clearThreshold()

SVM_correct = SVM_prediction_and_labels.filter(lambda predicted, actual: predicted == actual)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
model = DecisionTree.trainClassifier(trained_hashed, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
predictions = model.predict(test_hashed.map(lambda x: x.features))

labelsAndPredictions = test_hashed.map(lambda lp: lp.label).zip(predictions)

testErr = labelsAndPredictions.filter(lambda lp: lp[0] != lp[1]).count() / float(test_hashed.count())